In [1]:
from ETL import ETL
from modules.Transformation import Transformation
from modules.DataSource import DataSourceConfig
from modules.Encoder import EncoderConfig
from modules.Loader import LoaderConfig
from modules.Utils import Utils
from ETL import FlowConfig

# Datasource Config

url = "https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29"

# Flow Config

flow_config = FlowConfig()

flow1 = flow_config.create_flow(
    ID_datasource=f'Website Scraping {url}',
    ID_encoder='Basic HTML Tags',
    ID_ingestion=f'Scraping Ingestion for {url}',
    ID_control='1 Hour Update',
    ID_loader='Json File Loader on export/list_of_countries_nominal_capital.json')

flow_config.add_flow(flow1)

# Def Custom Scraping

def scraping(soap):
    data = []
    tables = soap.find_all('tbody')
    rows = tables[2].find_all('tr')
    for row in rows:
        col = row.find_all('td')
        if len(col)!=0:
            if col[0].find('a') is not None and '—' not in col[2]:
                data_dict = {"Country": col[0].a.contents[0],
                             "GDP_USD_millions": col[2].contents[0]}
                data.append(data_dict)
    return data

Utils().save_function('scraping_functions/extract_data_test.pkl', scraping) # I can save the scraping function

custom_scraping = Utils().load_function('scraping_functions/extract_data_test.pkl') # I can load a scraping function

# Pipeline Object Creation

pipeline = ETL("ETL Pipeline")

# Create Pipeline Elements

## Flow 1

pipeline.create_datasource(f'Website Scraping {url}', DataSourceConfig().scraping_source_config(url))
pipeline.create_encoder('Basic HTML Tags', EncoderConfig().html_encoder_config(['Country','GDP_USD_millions'], ['Country','GDP_USD_millions']))
pipeline.create_ingestion(f'Scraping Ingestion for {url}', custom_scraping)
pipeline.create_process("Transformation Basic for HTML", [Transformation().scraping_to_pivot_table])
pipeline.create_control('1 Hour Update',3600,3600)
pipeline.create_loader('Json File Loader on export/list_of_countries_nominal_capital.json', LoaderConfig().json_file_loader(f'export/list_of_countries_nominal_capital.json'))

# Set Pipeline flow

pipeline.set_flow(flow_config.get_flow_config())

# Pipeline Start 

pipeline.start()

Linking Pipeline flows
Flow index: 1 saved and ready to be run.


KeyError: 'Scraping Ingestion for https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'

In [1]:
import pandas as pd
from modules.Utils import Utils

df = pd.DataFrame(Utils().read_json('export/wikipedia.json'))

df[0:500]

,Country,GDP_USD_millions
0,United States,"26,854,599"
1,China,"19,373,586"
2,Japan,"4,409,738"
3,Germany,"4,308,854"
4,India,"3,736,882"
...,...,...
186,Marshall Islands,291
187,Palau,262
188,Kiribati,248
189,Nauru,151
